# Prepare environment

## Configure jupyter

* Install libs

In [ ]:
%%capture
!pip install cmake
!pip install deepface

In [ ]:
!pip install dlib

In [ ]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt

# run all code cells of the specified notebook
%run ../face/image_utils.ipynb

# Use DeepFace library

In [ ]:
from deepface import DeepFace

backend_names = ["opencv", "ssd", "dlib", "mtcnn", "retinaface", "mediapipe"]
model_names = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "ArcFace"]
# model_names = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "ArcFace", "DeepID", "Dlib", "SFace"]

target_size=(224, 224)

face_path = '/data/faces/'
person_path = face_path + 'elon_musk/'
face1 = person_path + 'image1.jpeg'
face2 = person_path + 'image2.jpeg'

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(15, 10))
axs = axs.flatten()

for index, backend_name in enumerate(backend_names):
    try:
        face = DeepFace.detectFace(face1, target_size=target_size, detector_backend=backend_name)
        axs[index].imshow(face)
        axs[index].set_title(backend_name)
        axs[index].axis('off')
    except:
        pass
plt.show()

In [ ]:
for model_name in model_names:
    result = DeepFace.verify(
        img1_path = face1,
        img2_path = face2,
        model_name=model_name
    )
    print(model_name, '=>', result)

    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle(f'Verified {result["verified"]} - Distance {result["distance"]:0.4}')
    
    axs[0].imshow(plt.imread(face1))
    axs[0].axis("off")

    axs[1].imshow(plt.imread(face2))
    axs[1].axis("off")
    plt.show()

In [ ]:
result = DeepFace.find(img_path=face1, db_path=face_path)
result

In [ ]:
def plot_img_emotion(img_path, emo_df, figsize=(15, 5)):
    img = cv2.cvtColor(img_path, cv2.COLOR_BGR2RGB)
    fig, axs = plt.subplots(1, 2, figsize=figsize)
    axs[0].axis("off")
    axs[0].imshow(img)
    emo_df.sort_values("prediction").plot(kind="barh", figsize=figsize, ax=axs[1])
    plt.tight_layout()
    plt.show()

In [ ]:
from glob import glob
imgs = glob(person_path + "*")

for img_path in imgs:
    print(img_path)
    img = cv2.imread(img_path)
    demo = DeepFace.analyze(img_path=img_path, 
                     detector_backend=backend_names[4])
    emo_df = pd.DataFrame(demo[0]["emotion"], index=[0]).T.rename(
        columns={0: "prediction"}
    )
    plot_img_emotion(img, emo_df)
    plt.show()